# Create dataset

In [ ]:
import pandas as pd
import os

In [4]:
dataPath = os.getcwd() + '/court case data/testdata/'
caseCount = len(os.listdir(dataPath))
data = []
try:
    os.remove(dataPath + ".DS_Store")
except:
    print("No file DS_Store")
for filename in os.listdir(dataPath):
    f = open(os.path.join(dataPath, filename), encoding='utf-8')
    data.append([filename.replace('.txt', ''), f.read()])

verdict_df = pd.DataFrame(data, columns=["id", "case text"])
cases_df = pd.read_csv('./court case data/testdata.csv')
merged_df = cases_df.join(verdict_df.set_index('id'), on='id', how='left')

merged_df["verdict_date"] = pd.to_datetime(merged_df["verdict_date"])
merged_df["publication_date"] = pd.to_datetime(merged_df["publication_date"])

No file DS_Store


In [5]:
merged_df

,id,verdict_date,publication_date,verdict_type,jurisdiction_type,case text
0,ECLI-NL-RBNNE-2021-5018,2021-01-10,2021-11-23,uitspraak,['Strafrecht'],\n\nRECHTBANK NOORD-NEDERLAND\nAfdeling strafr...
1,ECLI-NL-RBZUT-2003-AH9598,2003-03-06,2003-09-07,uitspraak,['Strafrecht'],\n\nRECHTBANK ZUTPHEN\nMeervoudige economische...
2,ECLI-NL-RBZWB-2020-2646,2020-06-23,2020-06-23,uitspraak,['Strafrecht'],\n\nRECHTBANK ZEELAND-WEST-BRABANT\n\nStrafrec...
3,ECLI-NL-GHAMS-2019-1601,2019-08-05,2019-07-26,uitspraak,['Strafrecht'],\n\nafdeling strafrecht\nparketnummer: 23-0017...
4,ECLI-NL-GHAMS-2019-1602,2019-08-05,2019-07-26,uitspraak,['Strafrecht'],\n\nafdeling strafrecht\nparketnummer: 23-0017...
...,...,...,...,...,...,...
18119,ECLI-NL-RBAMS-2013-1294,2013-01-29,2013-08-10,uitspraak,['Strafrecht'],\n\nRECHTBANK AMSTERDAM\n\n\nVONNIS\n\n \n\n13...
18120,ECLI-NL-PHR-2020-1106,2020-11-24,2020-11-24,conclusie,['Strafrecht'],\n\nPROCUREUR-GENERAAL\n\n\nBIJ DE\n\n\nHOGE R...
18121,ECLI-NL-GHAMS-2017-2618,2017-06-29,2017-05-07,uitspraak,['Strafrecht'],\n\n\nparketnummer: 23-001217-13\ndatum uitspr...
18122,ECLI-NL-RBAMS-2013-BZ0392,2013-01-29,2013-01-02,uitspraak,['Strafrecht'],\nRECHTBANK AMSTERDAM \nVONNIS \n\n13/529144-...


# Split documents

### Old

In [ ]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|mr|mevr|mvr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|nl)"
articles = "[artikel ][0-9][.][0-9]"

def split_into_sentences2(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(articles,"[artikelnummer]",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

### New

In [8]:
import re

In [9]:
def split_into_sentences(text):
    sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])|\n', text)
    sentences = [x for x in sentences if len(x) > 1]
    return sentences


Two ways of splitting the documents:
    For Word2Vec, we need sentences to be an array of words.
    For the rest, just the sentence is enough.

In [10]:
sentence_list_by_word = []
sentence_list = []

for i in range(len(merged_df)):
    doc = merged_df.iloc[i]['case text']
    sentences = split_into_sentences(doc)
    sentence_list.append(sentences)
    for j in sentences:
        word_list = [x for x in j.lower().rstrip().replace('.', '').split(' ') if len(x)>0]
        sentence_list_by_word.append(word_list)
        
print(len(sentence_list))
print(len(sentence_list_by_word))
# print(sentence_list[8])

18124
5107544


Testers:

In [ ]:
for i in sentence_list[:1]:
    for j in i:
        print(j, '\n')

In [ ]:
for i in sentence_list_by_word[:1]:
    for j in i:
        print(j, '\n')

# Create Word2Vec model

In [13]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

Create and save model

In [14]:
dutch_word2vec_model = Word2Vec(sentences=sentence_list_by_word, vector_size=100, window=5, min_count=1, workers=4)
dutch_word2vec_model.save("word2vec_dutch_court_cases.model")

Load model

In [15]:
dutch_word2vec_model = Word2Vec.load("word2vec_dutch_court_cases.model")

In [17]:
sims = dutch_word2vec_model.wv.most_similar('smokkel', topn=100)
print([i[0] for i in sims])
# print(sims)

['export', 'import', 'invoer', 'doorvoer', 'exporteren', 'aanvoer', 'verkoop', 'gesmokkelde', 'verhandeling', 'handel', 'drugssmokkel', 'invoeren', 'versnijding', 'transporteren', 'fabricage', 'terugwinproces', 'leveranciers', 'producenten', 'cocaïnehandel', 'importeren', 'versnijden', 'smokkelen', 'organisatoren', 'overdracht', 'aankoop', 'levering', 'afhalers', 'bronlanden', 'doorlating', 'drugstransporten', 'doorlevering', '(invoer', 'uithalen', 'leverancier', 'productie/verwerking', 'in-/uitvoer', 'distributie', 'verwerking', 'doorverkoop', 'produktie', 'overdacht', 'straathandel', 'onderkennen/herkennen', 'georganiseerde', 'dealen', 'transporten', 'hennephandel', '(synthetische)', 'verspreiding', 'aankopen', 'vermenging', 'synthetische', 'kweek', 'toestroom', 'drugshandel', 'koeriers', 'verkooppunten', 'vervaardiging', 'verkopers', 'verwerkingsproces', 'leveranties', 'productie', 'vervoerden[h]', "'dealpanden'", 'grootschalige', 'binnensmokkelen', 'binnenbrengen', 'productieketen'

### Create list of drugs, smuggle, quantity keywords with Word2Vec model

In [24]:
def create_word2vec_relevant_words(words, matches):
    word2vec_list = []
    for word in words:
        results = dutch_word2vec_model.wv.most_similar(word, topn=100)
        for i in results:
            word2vec_list.append(i[0])
            
    word2vec_list = list(set([i for i in word2vec_list if word2vec_list.count(i)>matches]))
    return word2vec_list

Drugs list:

In [31]:
list_of_drugs = ['xtc', 'mdma', 'cocaine', 'wiet', 'speed', 'bmk', 'pmk']
word2vec_drug_list = create_word2vec_relevant_words(list_of_drugs, 2)

print(len(word2vec_drug_list))
print(word2vec_drug_list)

78
['amfetamine;', '(met)amfetamine', 'cocaïne', 'oxazepam', 'mefedron', 'pure', 'xtc-tabletten', 'speed', 'mdma-poeder', 'metamfetamine', 'xtc-pillen', 'ketamine', 'marihuana', 'harddrugs', 'mdma-kristallen', 'cannabis', '2c-b', 'crystal', 'amfetamine', 'mdma/xtc', 'opium', 'amfetaminesulfaat', 'kristallen', 'cocaine', 'crack', 'methanol', 'pillen', 'amfetamine)', 'temazepam', 'hasjiesj', 'mdma;', 'manitol', 'methamfetamine', 'fenacetine', 'drugs', 'hash', 'amfetaminebase', 'zwavelzuur', 'methadon', 'paracetamol', 'amfetamineolie', 'eindproduct', 'hennep)', 'coke', 'xtc', 'lidocaïne', 'morfine', 'levamisol', 'poeder', 'mdma', 'pillen)', 'ecstasy', '(mdma)', 'platina', '34-methyleendioxymethamfetamine', 'oxycodon', 'ghb', 'mdma)', 'heroïne)', 'heroïne;', 'hasj', 'weed', 'cafeïne', 'gbl', 'hashish', 'sildenafil', 'hennep', 'heroïne', 'n-formylamfetamine', 'cocaïne;', 'cocaïne)', 'pep', 'mapa', 'amfetaminepasta', 'amfetaminen', 'azijnzuuranhydride', 'diazepam', 'lsd']


Smuggle keyword list:

In [32]:
list_of_smuggle_words = ['smokkel', 'invoer', 'uitvoer', 'import', 'export', 'transport']
word2vec_smuggle_list = create_word2vec_relevant_words(list_of_smuggle_words, 3)

print(len(word2vec_smuggle_list))
print(word2vec_smuggle_list)
word2vec_smuggle_list = word2vec_smuggle_list + list_of_smuggle_words 

39
['aankoop', 'leverantie', 'doorlevering', 'produktie', 'importeren', 'in-/uitvoer', 'export', 'bewerking', 'handel', 'transporten', 'bronland', 'levering', 'smokkel', 'verhandeling', 'invoer', 'hennepkweek', 'vervoerden[h]', 'leverancier', 'productie', 'verkoop', '(invoer', 'versnijding', 'fabricage', 'aanvoer', 'cocaïnehandel', 'kweek', 'organisatoren', 'invoeren', 'gesmokkelde', 'productie/verwerking', 'import', 'afzet', 'binnenhalen', 'transporteren', 'uithalen', 'exporteren', 'hennepteelt', 'vervaardiging', 'doorvoer']


Quantity keyword list:

In [37]:
list_of_quantity_words = ['tabletten', 'kilo', 'gram', 'pakketten']
word2vec_quantity_list = create_word2vec_relevant_words(list_of_quantity_words, 1)

print(len(word2vec_quantity_list))
print(word2vec_quantity_list)

87
['gr', 'vaatjes', 'gram10', 'sealbags', 'papertrips', 'pillen/tabletten', 'wikkels', 'sibutramine', 'drums', 'km/h', 'xtc-pillen', 'verpakkingen', 'mg', 'sealtjes', 'gripzakken', 'gram”', 'km/u', 'gram;', 'pakketten', 'kilo)', 'stuks', '(xtc-)pillen', 'planten)', 'bolletjes', 'ug/1', 'kilo', 'promille', 'gram)', 'kilogram;', 'brokjes', 'pillen', 'kg)', 'emmers', 'tabletten', 'joints', 'zegels', 'zakjes', 'cm)', 'papiertjes', 'liter', 'potten', 'tabletten/pillen', 'blikken', 'ml', 'slikkersbollen', 'mg)', 'seals', 'mg/ml', 'm²', 'watt', 'pakjes', 'liter)', 'potjes', 'dozen', 'blokken', 'mdma-pillen', 'percent', 'db(a)', 'ton', 'doosjes', 'plakken', 'kg', 'kilo;', 'balen', 'gram13', 'm2', 'ponypacks', 'tonnen', 'pilletjes', 'ampullen', 'kilogram', 'bollen', 'kilogram)', 'cachetjes', 'milliliter', 'snowseals', 'mg/m2', 'pallets', 'bakjes', 'km', 'gripzakjes', 'blokjes', '%', 'flessen', 'ng/ml', 'kg”', 'flacons']


Country list:

In [54]:
list_of_countries = ['duitsland', 'colombia']
word2vec_country_list = create_word2vec_relevant_words(list_of_countries, 1)

print(len(word2vec_country_list))
print(word2vec_country_list)

84
['frankrijk', 'hamburg', 'engeland', 'canada', 'curaçao', 'bulgarije', 'alicante', 'thailand', 'marokko', 'italië', 'tsjechië', 'servië', 'sevilla', 'nigeria', 'syrië', 'guayaquil', 'noorwegen', 'tanger', 'mexico', 'antwerpen', 'trinidad', 'portugal', 'venezuela', 'dubai', 'australië', 'afrika', 'londen', 'polen', 'brussel', 'paramaribo', 'chili', 'schotland', 'iran', 'natal', 'hongarije', 'lissabon', 'bosnië', 'oostenrijk', 'ecuador', 'groot-brittannië', 'oekraïne', 'istanbul', 'brazilië', 'argentinië', 'roemenië', 'turkije', 'jamaica', 'kroatië', 'kenia', 'parijs', 'ierland', 'denemarken', 'rabat', 'china', 'panama', 'ghana', 'slowakije', 'madrid', 'boedapest', 'belgie', 'malaga', 'suriname', 'zwitserland', 'congo', 'costa', 'rica', 'barcelona', 'luxemburg', 'kinshasa', 'griekenland', 'irak', 'peru', 'lima', 'belgië', 'berlijn', 'amerika', 'nederland', 'dominicaanse', 'europa', 'pakistan', 'spanje', 'zuid-amerika', '[land]', 'zweden']


# Create SpaCy model

In [39]:
import spacy
from spacy import displacy
# !python -m spacy download nl_core_news_md
nlp = spacy.load('nl_core_news_md')


In [40]:
my_file = open("drugs list.txt", "r", encoding='utf-8')
my_file = my_file.readlines()
drugs_list = []
for i in my_file:
    drugs_list.append(i.replace('\n', ''))

my_file = open("countries list.txt", "r", encoding='utf-8')
my_file = my_file.readlines()
countries_list = []
for i in my_file:
    countries_list.append(i.replace('\n', ''))
countries_list = countries_list + word2vec_country_list

In [42]:
complete_drugs_list = list(set(drugs_list + word2vec_drug_list))

In [43]:
def configure_spacy_model():
    # Create dict of drug pattern and quantity pattern
    pattern_list = []
    
    drugs_ent_list = []
    for i in complete_drugs_list:
        pattern_list.append({"label": "DRUG", "pattern": [{"lower": i}]})
    
#     quantity_rule = {"label": "QUANTITY", "pattern": [{"IS_DIGIT": True}, {"LOWER": "gram"}]}
#     pattern_list.append(quantity_rule)
    for i in word2vec_quantity_list:
        pattern_list.append({"label": "QUANTITY", "pattern": [{"IS_DIGIT": True}, {"LOWER": i}]})
        pattern_list.append({"label": "QUANTITY", "pattern": [{"ENT_TYPE": "CARDINAL"}, {"LOWER": i}]})
    
    for i in countries_list:
        pattern_list.append({"label": "GPE", "pattern": [{"lower": i.replace(' ', '').lower()}]})
    
    # Add drug and quantity rules to the model
    config = {
   "phrase_matcher_attr": None,
   "validate": True,
   "overwrite_ents": True,
   "ent_id_sep": "||",
    }
    ruler = nlp.add_pipe("entity_ruler", config=config)

    #List of Entities and Patterns
#     patterns = drugs_ent_list
    ruler.add_patterns(pattern_list)

In [44]:
configure_spacy_model()

# Select cases and chunks to keep

In [45]:
print(len(sentence_list) == len(merged_df))

True


For every case, split the sentences. If a sentence in a case contains a drug, a smuggle word, and a location: keep chunk and save to trafficking_df

In [108]:
relevant_chunk_list = []
ecli_list = []

for index, case in enumerate(sentence_list):
    chunk_list = []
    trafficking_related = False
    for chunk in case:
        if any(drug in chunk for drug in word2vec_drug_list) and any(smuggle_word in chunk for smuggle_word in word2vec_smuggle_list):
            ents = nlp(chunk).ents
            if any(ent.label_ == "GPE" or ent.label_ == "LOC" for ent in ents):
                trafficking_related = True
                chunk_list.append(chunk)
    if trafficking_related:
        relevant_chunk_list.append(chunk_list)
        ecli_list.append(merged_df.iloc[index]['id'].replace('-', ':'))

trafficking_df = pd.DataFrame({'id': pd.Series(ecli_list), 'chunks': pd.Series(relevant_chunk_list)})                

In [48]:
print(f"{len(trafficking_df)} cases kept from original {len(merged_df)} cases.")


4670 cases kept from original 18124 cases.


# Create rule-based NER & POS tagging model

### Old

In [249]:
def extract_chunk_info(txt):
    source_country = None
    total_info = []
    for token in nlp(txt):
        info = {}
        drug_info = {}
        countries = []
        
        if token.ent_type_ == "DRUG":
            info = {"drug": token.text}
            
            ## Get source and destination
            for ancestor in token.ancestors:
                for nephew in ancestor.children:
                    if nephew.ent_type_ == "GPE" or nephew.ent_type_ == "LOC":
                        countries.append(nephew)
                        for child in nephew.children:
                            if child.dep_ == "conj" and child.ent_type_ == "GPE" or child.ent_type_ == "LOC":
                                countries.append(child.text)
                            elif child.pos_ == "ADP" and child.dep_ == "case":
                                adj = child.text
            if len(countries) > 0 :
                try:
                    info[adj] = countries
                except:
                    info['land'] = countries
                        
            ## Get volume
            for ancestors in token.ancestors:
                for nephew in ancestors.children:
                    if nephew.ent_type_ == "QUANTITY" or nephew.ent_type_ == "CARDINAL":
                        for second_nephew in nephew.children:
                            if second_nephew.is_digit != nephew.is_digit:
                                if second_nephew.is_digit:
                                    info['volume'] = second_nephew.text
                                    info['volume_type'] = nephew.text
                                else:
                                    info['volume'] = nephew.text
                                    info['volume_type'] = second_nephew.text
            if 'volume' not in info:
                for child in token.children:
                    if (child.dep_ == "det" and child.like_num) or (child.dep_ == "nummod"):
                        info['volume'] = child.text
                                
        if len(info) > 1:
#             print(info)
            total_info.append(info)
    return total_info

### New

In [156]:
# Get linguistic distance between token a and token b. After iter 10 it is deemed a too far distance.
def get_linguistic_distance(a, b):
    tokens_to_consider = [b]
    found = False
    iters = 0
    while not found:
        for token in tokens_to_consider:
            tokens_to_add = []
            for ancestor in token.ancestors:
                if ancestor not in tokens_to_add and ancestor not in tokens_to_consider:
                    tokens_to_add.append(ancestor)
            for child in token.children:
                if child not in tokens_to_add and child not in tokens_to_consider:
                    tokens_to_add.append(child)
            tokens_to_consider = tokens_to_consider + tokens_to_add
        for x in tokens_to_consider:
            if a.orth == x.orth:
                found = True
        iters += 1
        if iters == 10:
            found = True
    return iters

In [50]:
def get_adposition_from_loc(token):
    for child in token.children:
        if child.pos_ == "ADP" and child.dep_ == "case":
            return child.text

In [157]:
def extract_chunk_info(txt):
    result = {}
#     print('START CHUNK')
    for token in nlp(txt):
        if token.ent_type_ == "DRUG":
#             print(f" \n Extracting info for {token.text}.")
            drug_info = extract_info_from_drug(token, txt)
            result[token.text] = drug_info
    return result
            
            
def extract_info_from_drug(drug, txt):
    volumes = []
    locations = {}
    irrelevant_locations = []
    for token in nlp(txt):
        
        # Extract countries
        if token.ent_type_ == "GPE" or token.ent_type_ == "LOC":
            dist = get_linguistic_distance(drug, token)
            if dist < 10:
                adj = get_adposition_from_loc(token)
#                 print(f"    {adj}: {token.text}, dist: {dist}, conj: {token.conjuncts}")
                locs = [token.text]
                for loc in token.conjuncts:
                    locs.append(loc.text)
                locations[adj] = locs
            else:
                irrelevant_locations.append(token.text)
#                 print(f"{token.text} is irrelevant.")
        
        # Extract volume
        if token.ent_type_ == "QUANTITY":
            volume = {}
            dist = get_linguistic_distance(drug, token)
            second_token = ""
            if dist < 10:
                quantity = {}
                for ancestor in token.ancestors:
                    if ancestor.ent_type_ == "QUANTITY":
                        second_token = ancestor
                for child in token.children:
                    if child.ent_type_ == "QUANTITY":
                        second_token = child
                
                ## Decide volume and volume_type
                if nlp(token.text)[0].ent_type_ == "CARDINAL":
                    volume['volume'] = token.text
                    volume['volume_type'] = second_token.text
                elif nlp(second_token.text)[0].ent_type_ == "CARDINAL":
                    volume['volume'] = second_token.text
                    volume['volume_type'] = token.text
                
                #Only append when not already in volumes
                if volume not in volumes:
                    volumes.append(volume)
#             else:
#                 print(f"{token.text} is irrelevant.")
                
        
        
            
#     print(volumes)

    result = {}
    if bool(locations):
        result['locations'] = locations
    if len(volumes) > 0:
        result["volume"] = volumes
    if len(irrelevant_locations) > 0:
        result["irrelevant_locations"] = irrelevant_locations
    
    return result
    
        


# Results

In [160]:
def get_summary(ecli):
    rel_chunks = trafficking_df[trafficking_df['id'] == ecli]['chunks'].tolist()[0]
    print(f"{len(rel_chunks)} chunks in ecli: {ecli}")
    for chunk in rel_chunks:
        info = extract_chunk_info(chunk)
        drugtype = next(iter(info))
        if 'locations' in info[drugtype]:
            print(info)
        
get_summary('ECLI:NL:GHARN:2004:AR3598')

IndexError: list index out of range

In [159]:
from_case = 40
to_case = 60

result_list = []

for index in range(len(trafficking_df[:from_case+to_case])):
    if index >= from_case:
        ecli = trafficking_df.iloc[index]['id']
        get_summary(ecli)
        print('\n')


9 chunks in ecli: ECLI:NL:RBMAA:2010:BM8942
{'amfetamine': {'locations': {'te': ['Landgraaf']}, 'volume': [{}]}}
{'wiet': {'locations': {'naar': ['Duitsland']}}, 'amfetamine': {'locations': {'naar': ['Duitsland']}}}
{'amfetamine': {'locations': {None: ['Bonn']}, 'volume': [{'volume': 'vijftig', 'volume_type': 'kilo'}]}}


StopIteration: 

In [689]:
for i in result_list:
    print(i)

{'locations': {'te': ['Landgraaf']}, 'volume': [{}], 'amfetamine': {...}}
{'locations': {'naar': ['Duitsland']}, 'amfetamine': {...}}
{'locations': {None: ['Bonn']}, 'volume': [{'volume': 'vijftig', 'volume_type': 'kilo'}], 'amfetamine': {...}}
{}
{'locations': {'naar': ['Duitsland']}, 'amfetamine': {...}}
{'locations': {'te': ['Landgraaf']}, 'amfetamine': {...}}
{'locations': {'naar': ['Duitsland']}, 'wiet': {...}}
{'locations': {'te': ['Landgraaf']}, 'amfetamine': {...}}
{'locations': {'te': ['Landgraaf']}, 'amfetamine': {...}}
{'locations': {'te': ['Landgraaf']}, 'volume': [{}], 'amfetamine': {...}}
{'locations': {'naar': ['Duitsland']}, 'amfetamine': {...}}
{'locations': {'te': ['Landgraaf']}, 'amfetamine': {...}}
{'locations': {'in': ['Duitsland']}, 'amfetamine': {...}}
{'volume': [{'volume': 'vijftig', 'volume_type': 'kilo'}], 'amfetamine': {...}}
{}
{'locations': {'naar': ['Duitsland']}, 'amfetamine': {...}}
{'locations': {'te': ['Landgraaf']}, 'amfetamine': {...}}
{'locations':